In [ ]:
pip install -qU langchain_community tiktoken langchain-openai langchainhub chromadb langchain langgraph json dotenv

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')
os.environ['TAVILY_API_KEY']= os.getenv('TAVILY_API_KEY')

# 예산 관리 자유 질문

In [2]:
import json

file_path = 'data.json'  

with open(file_path, 'r', encoding='utf-8') as file:
    data = json.load(file)

In [3]:
def extract_spending_details_by_category(data):
    categories = ['본식', '스드메', '예복', '예물/예단', '신혼여행', '가전/가구', '기타']
    
    result = {category: [] for category in categories}
    
    for category_data in data['groupedSpendings']:
        category_name = category_data['budgetCategory']['budgetCategoryName']
        # totalAmount은 사용 x
        #ipdb.set_trace()
        
        if category_name in categories:
            for spending in category_data['spendings']:
                spending_details = {
                    f"{spending['title']}": spending['amount']
                }
                result[category_name].append(spending_details)
    
    return result


extracted_budget_data = extract_spending_details_by_category(data)

print(extracted_budget_data)

{'본식': [{'식대': 0}, {'홀 대관료': 1000000}], '스드메': [{'스드메': 1439000}], '예복': [{'예복': 0}], '예물/예단': [{'결혼반지': 2390000}], '신혼여행': [{'숙박비': 0}, {'항공권': 0}, {'투어 및 기타 경비': 0}], '가전/가구': [{'청소기': 759000}, {'침대': 8102000}, {'삼성': 8770000}, {'식탁': 722000}, {'쇼파': 3204820}], '기타': [{'줄눈': 450000}, {'탄성코트': 550000}, {'하자점검': 270000}, {'입주청소': 510000}, {'TV(무타공)': 330000}]}


In [11]:
# English 
PROMPT_TEMPLATE_EG = """
# Instruction 
You are an AI assistant helping with wedding budget management. Please respond to the user's questions kindly and accurately. Your goal is to provide helpful advice based on the user’s current wedding budget, suggest possible savings or cost-efficient alternatives, and, when needed, provide additional information by searching the internet for price comparisons, recommendations, or tips.

Current wedding budget information:
{budget_info}


# GUIDELINES:

1. Always consider the user’s wedding budget limitations and provide cost-efficient suggestions where possible.
2. When there are zero values or missing information in certain categories, offer advice on possible cost ranges or alternatives based on typical wedding expenditures.
3. For large expenses, help the user identify areas where they might save by comparing current prices online or recommending alternative options (e.g., for furniture or honeymoon).
4. Only respond to queries that are directly related to wedding budget management or planning.  
5. Keep your tone friendly, empathetic, and professional.
6. The user is Korean. Provide all information based on Korean wedding standards.

# IMPORTANT : Answer in KOREAN

# Begin!

Previous conversation history:
{chat_history}

New input: {input}

"""

In [6]:
# 한국어 
PROMPT_TEMPLATE_KR = """
# 지침 
당신은 결혼 예산 관리를 돕는 AI 비서입니다. 사용자 질문에 친절하고 정확하게 답변해 주세요. 목표는 사용자의 현재 결혼 예산 정보를 기반으로 유용한 조언을 제공하고, 가능한 절약 방법이나 비용 효율적인 대안을 제안하는 것입니다. 필요할 경우 인터넷 검색을 통해 가격 비교, 추천 또는 팁을 추가로 제공하세요.

현재 결혼 예산 정보:
{budget_info}


# 가이드라인:
1. 항상 사용자의 결혼 예산 제한을 고려하고, 가능한 비용 효율적인 제안을 제공하세요.
2. 특정 항목에서 값이 없거나 정보가 부족한 경우, 일반적인 결혼 지출 범위를 바탕으로 가능한 비용 범위나 대안을 제시하세요.
3. 큰 비용 항목에 대해서는 현재 가격을 비교하거나 대체 옵션(예: 가구 또는 신혼여행)을 추천하여 절약할 수 있는 영역을 사용자에게 안내하세요.
4. 결혼 예산 관리 또는 계획과 직접적으로 관련된 질문에만 답변하세요.
5. 친근하고 전문적인 어조를 유지하세요.
6. 사용자는 한국인입니다. 모든 정보는 한국 결혼 기준으로 답변하세요.

# 중요: 
답변은 반드시 한국어로 작성하세요.

# 시작!
이전 대화 기록:
{chat_history}

새로운 입력: {input}

"""


In [15]:
import json

from langchain.chains import LLMChain
from langchain_openai import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate


class BudgetAssistant:
    def __init__(self, budget_data):
        """
        예산 어시스턴트 초기화
        
        Args:
            budget_data (dict): 사용자의 예산 정보
        """
        
        self.budget_data = budget_data
        self.memory = ConversationBufferMemory(
            memory_key="chat_history",
            input_key="input"
        )
        
        self.prompt = PromptTemplate(
            input_variables=["chat_history", "input", "budget_info"],
            template=PROMPT_TEMPLATE_EG # 프롬프트 변경
        )
        
        self.llm = ChatOpenAI(
            model="gpt-4o-mini",
            temperature=0.5 #temperature 조절
    
        )
        
        self.chain = LLMChain(
            llm=self.llm,
            prompt=self.prompt,
            memory=self.memory
        )
    
    def ask(self, question):
        """
        질문에 대한 응답 생성
        
        Args:
            question (str): 사용자 질문
        
        Returns:
            str: LLM의 응답
        """
        return self.chain.run(
            input=question,
            budget_info=json.dumps(self.budget_data, ensure_ascii=False, indent=2)
        )

def main():
    
    # 예산 데이터
    extracted_budget_data = extract_spending_details_by_category(data)

    # 예산 어시스턴트 초기화
    assistant = BudgetAssistant(extracted_budget_data)
    
    print("예산 관리 어시스턴트입니다. 질문해 주세요! (종료하려면 'quit' 입력)")
    
    while True:
        user_input = input("\n질문: ")
        
        if user_input.lower() == 'quit':
            print("상담을 종료합니다. 감사합니다!")
            break
            
        try:
            response = assistant.ask(user_input)
            print("\n답변:", response)
        except Exception as e:
            print(f"\n오류가 발생했습니다: {str(e)}")

if __name__ == "__main__":
    main()

예산 관리 어시스턴트입니다. 질문해 주세요! (종료하려면 'quit' 입력)

답변: 안녕하세요! 결혼 예산에 대해 조언을 드릴 수 있어서 기쁩니다. 현재 예산을 살펴보면 몇 가지 항목에서 비용이 발생하고 있지만, 일부 항목은 아직 예산이 설정되지 않았네요. 아래는 각 항목에 대한 조언입니다.

1. **본식 식대**: 식대는 결혼식에서 가장 큰 비용 중 하나입니다. 일반적으로 1인당 5만 원에서 10만 원 정도 예상할 수 있습니다. 예를 들어, 100명을 초대하면 500만 원에서 1000만 원 정도의 예산이 필요할 수 있습니다. 예산을 세우실 때 이 점을 고려해 보세요.

2. **예복**: 현재 예복에 대한 예산이 설정되어 있지 않네요. 대여를 고려하신다면 대여 비용은 보통 50만 원에서 150만 원 정도입니다. 예복을 구매하신다면 100만 원 이상의 예산을 생각하셔야 할 것입니다.

3. **신혼여행**: 현재 신혼여행에 대한 예산이 전혀 설정되어 있지 않네요. 평균적으로 신혼여행 비용은 200만 원에서 500만 원 정도로 예상할 수 있습니다. 국내 여행을 고려하신다면 비용을 절약할 수 있습니다.

4. **가전/가구**: 가전제품과 가구의 경우, 큰 비용이 발생하고 있습니다. 특히 침대와 삼성 제품의 가격이 상당히 높은데, 중고 제품이나 할인된 제품을 고려하시면 예산을 절약할 수 있습니다. 또한, 가전제품을 패키지로 구매하면 할인 혜택을 받을 수 있으니 확인해 보세요.

5. **기타 항목**: 기타 비용 항목도 예산을 잘 세워야 합니다. 필요 없는 항목은 줄이고, 꼭 필요한 것만 선택하는 것이 좋습니다.

예산을 조정할 수 있는 부분이나 더 궁금한 점이 있으시면 언제든지 말씀해 주세요! 도움이 필요하시면 언제든지 도와드리겠습니다.

답변: 안녕하세요! 결혼 예산을 정리해드리겠습니다. 현재 설정된 항목들을 기반으로 예산을 요약해보면 다음과 같습니다.

### 결혼 예산 요약

1. **본식**
   - 식대: 0원 (추가 필요)
   - 홀 대관료: 1